In [2]:
# 1. Import the l
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random
import os

Using TensorFlow backend.
/Users/kudokhang/opt/miniconda3/envs/jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kudokhang/opt/miniconda3/envs/jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kudokhang/opt/miniconda3/envs/jupyter/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [3]:
# 2. Define image properties
Image_W, Image_H, Image_C = 128,128,3
Image_Size = (Image_W, Image_H)

In [4]:
# 3. Prepare datasets for traing model
filenames = os.listdir("./dogs-vs-cats/train")
categories = []
for f_name in filenames:
    category = f_name.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)
    
df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [3]:
# 4. Create the neural net model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

model = Sequential()

In [6]:
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_W,Image_H,Image_C)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [7]:
# 5. Analyzing model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_2 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)       

In [8]:
# 6. Define callbacks and laerning rate
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [9]:
# 7. Manage data
df["category"] = df["category"].replace({0:'cat', 1:'dog'})
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size = 15

In [10]:
# 8. Training and validation data generator
train_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1)
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                   "./dogs-vs-cats/train",
                                                   x_col='filename',
                                                   y_col='category',
                                                   target_size=Image_Size,
                                                   class_mode='categorical',
                                                   batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                             "./dogs-vs-cats/train",
                                                             x_col='filename',
                                                             y_col='category',
                                                             target_size=Image_Size,
                                                             class_mode='categorical',
                                                             batch_size=batch_size)


test_datagen = ImageDataGenerator(rotation_range=15,
                                 rescale=1./255,
                                 shear_range=0.1,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1)
test_generator = test_datagen.flow_from_dataframe(train_df,
                                                 "./dogs-vs-cats/test1",
                                                 x_col='filename',
                                                 y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Found 0 validated image filenames belonging to 0 classes.


/Users/kudokhang/opt/miniconda3/envs/jupyter/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 20000 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [18]:
# 9. Model training
epochs = 10
history = model.fit_generator(train_generator,
                             epochs=epochs,
                             validation_data=validation_generator,
                             validation_steps=total_validate//batch_size,
                             steps_per_epoch=total_train//batch_size,
                             callbacks=callbacks)


Epoch 1/10
1333/1333 [==============================] - 1472s 1s/step - loss: 0.7505 - accuracy: 0.6275 - val_loss: 0.6331 - val_accuracy: 0.7015
Epoch 2/10


/Users/kudokhang/opt/miniconda3/envs/jupyter/lib/python3.7/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1045/1333 [======================>.......] - ETA: 4:53 - loss: 0.5629 - accuracy: 0.7162

KeyboardInterrupt: 

In [4]:
model.save("model1_catsVSdogs.h5")

NotImplementedError: 

In [5]:
# 11. Test data preparation
test_filenames = os.listdir("./dogs-vs-cats/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [7]:
from keras.models import load_model
model = load_model('./model1_catsVSdogs.h5')

In [9]:
# 12. Make categorical prediction
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

NameError: name 'test_generator' is not defined

In [10]:
# 13. Convert labels to categories
test_df['category'] = np.argmax(predict, exis=-1)
labels_map = dict((v,k) for k, v in train_generator.class_indices.items())
test_df["category"] = test_df["category"].replace(label_map)

test_df["category"] = test_df["category"].replace({
    'dog':1,'cat':0
})



NameError: name 'predict' is not defined

In [ ]:
# 14. Visualize the prediction results
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("./dogs-vs-cats/test1/"+filename, target_size=Image_Size)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()